In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time


hotel_data = []
# Initialize the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Function to scrape individual hotel data
def scrape_hotel_data(hotel_url):
    driver.get(hotel_url)
    driver.implicitly_wait(10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Extract the hotel name
    hotel_name_tag = soup.find('span', class_='a5H0ec')
    hotel_name = hotel_name_tag.text if hotel_name_tag else 'N/A'
    
    # Extract the review score
    review_score_tag = soup.find('div', class_='F7nice')
    review_score = review_score_tag.text if review_score_tag else 'N/A'
    
    # Extract the price
    price_tag = soup.find('span', class_='fontTitleLarge Cbys4b')
    price = price_tag.text if price_tag else 'N/A'
    
    # Extract the address
    address_button_tag = soup.find('button', class_='CsEnBe')
    address = address_button_tag.get('aria-label') if address_button_tag else 'N/A'
    
    return hotel_name, address, price, review_score

# Open the Google Maps page with hotels near Lahore
driver.get('https://www.google.com/maps/search/hotels+near+lahore')
driver.implicitly_wait(10)

# Extract the page source after JavaScript execution
page_source = driver.page_source

# Use BeautifulSoup to parse the dynamic content
soup = BeautifulSoup(page_source, 'html.parser')

# Find the relevant data for the first 5 hotels
hotels = soup.find_all('a', class_='hfpxzc', limit=5)

for hotel in hotels:
    # Extract hotel name from 'aria-label'
    hotel_name = hotel.get('aria-label')
    
    # Extract link from 'href'
    link = hotel.get('href')
    hotel_link = f'{link}'
    
    # Scrape individual hotel data
    name, address, price, review_score = scrape_hotel_data(hotel_link)
    
    # Print the details in the desired format
    print(f'hotel_name: {hotel_name}')
    print(f'hotel_link: {hotel_link}')
    print(f'hotel_address: {address}')
    print(f'hotel_price: {price}')
    print(f'hotel_review_score: {review_score}')
    print('------')
    
    hotel_data.append({
        'hotel_name': hotel_name,
        'hotel_link': hotel_link,
        'hotel_address': address,
        'hotel_price': price,
        'hotel_review_score': review_score
    })

    
    # Go back to the hotel list page
    driver.back()
    driver.implicitly_wait(10)
    time.sleep(2)  # Adding a delay to ensure the page loads properly
    
df = pd.DataFrame(hotel_data)

# Close the driver
driver.quit()


hotel_name: Taj Palace Hotel
hotel_link: https://www.google.com/maps/place/Taj+Palace+Hotel/data=!4m10!3m9!1s0x391903faa7a7ed87:0x6b9261a2d964ed59!5m2!4m1!1i2!8m2!3d31.5336084!4d74.2824451!16s%2Fg%2F11h2jv34m2!19sChIJh-2np_oDGTkRWe1k2aJhkms?authuser=0&hl=en&rclk=1
hotel_address: Address: bund road, near rahber travel(Bilal travel, opp. city bus terminal, Dholanwal Lahore, Punjab 54000 
hotel_price: Rs 2,146
hotel_review_score: 3.9(955)
------
hotel_name: Hotel Days INN Lahore
hotel_link: https://www.google.com/maps/place/Hotel+Days+INN+Lahore/data=!4m10!3m9!1s0x3919017aea68437d:0x6176c8dc5ec06945!5m2!4m1!1i2!8m2!3d31.4601405!4d74.2604996!16s%2Fg%2F11h3pt0fz7!19sChIJfUNo6noBGTkRRWnAXtzIdmE?authuser=0&hl=en&rclk=1
hotel_address: Address: Block-2 Sher Ali Road opposite Expo Center gate, St 7, Lahore 
hotel_price: N/A
hotel_review_score: 3.9(460)
------
hotel_name: Grand Swiss Hotel&Apartment's Lahore
hotel_link: https://www.google.com/maps/place/Grand+Swiss+Hotel%26Apartment%27s+Lahore/da

In [23]:
df

,hotel_name,hotel_link,hotel_address,hotel_price,hotel_review_score
0,Taj Palace Hotel,https://www.google.com/maps/place/Taj+Palace+H...,"Address: bund road, near rahber travel(Bilal t...","Rs 2,146",3.9(955)
1,Hotel Days INN Lahore,https://www.google.com/maps/place/Hotel+Days+I...,Address: Block-2 Sher Ali Road opposite Expo C...,N/A,3.9(460)
2,Grand Swiss Hotel&Apartment's Lahore,https://www.google.com/maps/place/Grand+Swiss+...,"Address: MA, 756-G4 Fatimiyya Rd, Block G 4 Ph...","Rs 5,481",4.3(224)
3,Hotel Days Inn Emporium,https://www.google.com/maps/place/Hotel+Days+I...,"Address: 259 H, Mall, Emporium, Block H-2 Bloc...","Rs 2,584",3.7(223)
4,Hotel One Gulberg,https://www.google.com/maps/place/Hotel+One+Gu...,"Address: A, Ali Kasuri 40, 2 Lajpat Rd, Lahore",N/A,"3.9(2,173)"


In [21]:
df.to_csv('hotels_info.csv', index=False)